## Declare Azure LLM

In [1]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",  
    api_version="2024-08-01-preview", 
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [2]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain.graphs import Neo4jGraph

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris. 
"""
print(len(text))

575


In [ ]:
# documents = [Document(page_content=text)]
# llm_transformer = LLMGraphTransformer(llm=llm)
# graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
# print(f"Nodes:{graph_documents[0].nodes}")
# print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person'), Node(id='1867', type='Year'), Node(id='Polish', type='Nationality'), Node(id='French', type='Nationality'), Node(id='Physicist', type='Occupation'), Node(id='Chemist', type='Occupation'), Node(id='Radioactivity', type='Field of study'), Node(id='Nobel Prize', type='Award'), Node(id='First Woman', type='Achievement'), Node(id='First Person', type='Achievement'), Node(id='Two Scientific Fields', type='Achievement'), Node(id='Pierre Curie', type='Person'), Node(id='Married Couple', type='Relationship'), Node(id='University Of Paris', type='University')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='1867', type='Year'), type='BORN_IN'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Polish', type='Nationality'), type='HAS_NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='French', type='Nationality'), type='HAS_NATIONALITY'), Relations

In [ ]:
graph = Neo4jGraph()

In [ ]:
# graph.add_graph_documents(
#     graph_documents,
#     baseEntityLabel=True,
#     include_source=True
# )

### Querying the Graph Database

In [4]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain

CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to query a graph database.
...
"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
...
"""
CYPHER_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

graph_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    # cypher_prompt=CYPHER_GENERATION_PROMPT, 
    # qa_prompt=CYPHER_QA_PROMPT,
    verbose=True
)


In [5]:
question = """Who is Marie Curie?"""
graph_chain.run(question)

/home/nikunj/Downloads/IRE/Project/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
To generate a Cypher statement to query a graph database, you need to specify the pattern of nodes and relationships you want to match in the graph. Here's an example of a Cypher statement:

MATCH (n:Person)-[:FRIEND]->(m:Person)
WHERE n.name = 'John'
RETURN m.name

In this example, the statement matches two nodes labeled "Person" connected by a "FRIEND" relationship. It then filters the results to only include nodes where the property "name" of the first node is equal to 'John'. Finally, it returns the "name" property of the second node.

You can modify this statement to match your specific graph structure and query requirements.


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'To': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"To generate a Cypher statement to query a graph database, you need to specify the pattern of nodes and relationships you want to match in the graph. Here's an example of a Cypher statement:"
 ^}